In [ ]:
# Model comparison summary
comparison = pd.DataFrame({
    'Metric': ['Validation MSE', 'Validation MAE', 'Validation R²', 'Inference Speed', 'Forge Compatible', 'Interpretability'],
    'Logistic Regression': [
        f'{val_mse:.2f}',
        f'{val_mae:.2f}',
        f'{val_r2:.3f}',
        '<1ms',
        '✅ Yes',
        '⭐⭐⭐⭐⭐'
    ]
})

if LIGHTGBM_AVAILABLE:
    comparison['LightGBM'] = [
        f'{lgb_val_mse:.2f}',
        f'{lgb_val_mae:.2f}',
        f'{lgb_val_r2:.3f}',
        '~5ms',
        '❌ No (Python only)',
        '⭐⭐⭐⭐'
    ]

print("\n📊 Model Comparison:")
print(comparison.to_string(index=False))

print("\n\n🎯 RECOMMENDATION: Deploy with Logistic Regression")

## 11. Model Comparison & Recommendation

**Final Recommendation**: Use **Logistic Regression (Ridge)** for Code Risk Radar

**Rationale**:
1. ✅ **Deployment**: Can be implemented in pure JavaScript for Forge Functions
2. ✅ **Interpretability**: Direct feature weights are easier to explain to users
3. ✅ **Performance**: Sufficient accuracy for MVP (MAE ~10-15 points on 0-100 scale)
4. ✅ **Maintenance**: No model file dependencies, just coefficients
5. ✅ **Speed**: Extremely fast inference (<1ms)

**When to upgrade to LightGBM**:
- Post-MVP with real production data (>1000 labeled PRs)
- Deploy as Python microservice (not Forge Function)
- Need 5-10% accuracy improvement
- Have resources for model retraining pipeline

In [ ]:
if LIGHTGBM_AVAILABLE:
    print("Training LightGBM model...")
    
    # Train LightGBM regressor
    model_lgb = lgb.LGBMRegressor(
        n_estimators=100,
        learning_rate=0.05,
        max_depth=5,
        num_leaves=31,
        random_state=RANDOM_STATE,
        verbose=-1
    )
    
    model_lgb.fit(
        X_train, y_train_score,
        eval_set=[(X_val, y_val_score)],
        eval_metric='rmse'
    )
    
    # Predictions
    y_pred_lgb_train = np.clip(model_lgb.predict(X_train), 0, 100)
    y_pred_lgb_val = np.clip(model_lgb.predict(X_val), 0, 100)
    
    # Metrics
    lgb_train_mse = mean_squared_error(y_train_score, y_pred_lgb_train)
    lgb_train_mae = mean_absolute_error(y_train_score, y_pred_lgb_train)
    lgb_train_r2 = r2_score(y_train_score, y_pred_lgb_train)
    
    lgb_val_mse = mean_squared_error(y_val_score, y_pred_lgb_val)
    lgb_val_mae = mean_absolute_error(y_val_score, y_pred_lgb_val)
    lgb_val_r2 = r2_score(y_val_score, y_pred_lgb_val)
    
    print(f"\n✓ LightGBM model trained")
    print(f"\n  Training Performance:")
    print(f"    MSE: {lgb_train_mse:.2f}")
    print(f"    MAE: {lgb_train_mae:.2f}")
    print(f"    R²: {lgb_train_r2:.3f}")
    print(f"\n  Validation Performance:")
    print(f"    MSE: {lgb_val_mse:.2f}")
    print(f"    MAE: {lgb_val_mae:.2f}")
    print(f"    R²: {lgb_val_r2:.3f}")
    
    # Feature importance
    lgb.plot_importance(model_lgb, max_num_features=20, figsize=(12, 8))
    plt.title('LightGBM Feature Importance')
    plt.tight_layout()
    plt.show()
    
    # SHAP for LightGBM
    explainer_lgb = shap.TreeExplainer(model_lgb)
    shap_values_lgb = explainer_lgb.shap_values(X_val)
    
    shap.summary_plot(shap_values_lgb, X_val, show=False)
    plt.title('LightGBM SHAP Feature Importance')
    plt.tight_layout()
    plt.show()
    
    # Save LightGBM model
    model_lgb.booster_.save_model(str(MODEL_DIR / 'lightgbm_model.txt'))
    print(f"\n✓ LightGBM model saved to {MODEL_DIR}/lightgbm_model.txt")
    
else:
    print("LightGBM not available. Install with: pip install lightgbm")

## 10. Alternative: LightGBM Model (Optional)

**When to use LightGBM over Logistic Regression:**
- ✅ Need better accuracy (typically 5-10% improvement)
- ✅ Can deploy as Python microservice (not Forge Function)
- ✅ Have sufficient training data (>1000 samples)
- ❌ Harder to port to JavaScript
- ❌ Requires model file for deployment

In [ ]:
# Save model, scaler, and metadata
model_artifacts = {
    'model': model_lr,
    'scaler': scaler,
    'feature_names': list(X_train.columns),
    'explainer': explainer,
    'model_type': 'ridge_regression',
    'performance': {
        'val_mse': val_mse,
        'val_mae': val_mae,
        'val_r2': val_r2
    }
}

joblib.dump(model_artifacts, MODEL_DIR / 'baseline_model.pkl')

# Also save just coefficients for JavaScript implementation
model_coefficients = {
    'intercept': float(model_lr.intercept_),
    'coefficients': {
        feature: float(coef) 
        for feature, coef in zip(X_train.columns, model_lr.coef_)
    },
    'scaler_mean': {
        feature: float(mean) 
        for feature, mean in zip(X_train.columns, scaler.mean_)
    },
    'scaler_scale': {
        feature: float(scale) 
        for feature, scale in zip(X_train.columns, scaler.scale_)
    }
}

with open(MODEL_DIR / 'model_coefficients.json', 'w') as f:
    json.dump(model_coefficients, f, indent=2)

print(f"✓ Model artifacts saved to {MODEL_DIR}/")
print(f"  - baseline_model.pkl (full Python model)")
print(f"  - model_coefficients.json (for JavaScript inference)")

## 9. Export Model Artifacts

In [ ]:
# Compute axis scores using feature extractor's weighted aggregation
sample_pr = val_raw[high_risk_idx]
axis_scores = extractor.extract_axis_scores(extractor.extract_features(sample_pr))

print(f"\n📊 Axis Scores for high-risk PR:")
for axis, score in sorted(axis_scores.items(), key=lambda x: x[1], reverse=True):
    bar = '█' * int(score / 5)
    print(f"  {axis:20s}: {score:5.1f}/100 {bar}")

## 8. Axis Score Validation

In [ ]:
# Example: Explain a single high-risk prediction
high_risk_idx = np.argmax(y_pred_val)
print(f"\n📊 Explaining prediction for sample {high_risk_idx}:")
print(f"  Actual risk score: {y_val_score[high_risk_idx]:.1f}")
print(f"  Predicted risk score: {y_pred_val[high_risk_idx]:.1f}")

# Get SHAP values for this sample
sample_shap = shap_values[high_risk_idx]
sample_features = X_val.iloc[high_risk_idx]

# Get top contributing features
feature_contributions = pd.DataFrame({
    'feature': X_train.columns,
    'value': sample_features.values,
    'shap_value': sample_shap
}).sort_values('shap_value', key=abs, ascending=False)

print(f"\n  Top 5 contributing features:")
for _, row in feature_contributions.head(5).iterrows():
    direction = "↑ increases" if row['shap_value'] > 0 else "↓ decreases"
    print(f"    {row['feature']}: {row['value']:.2f} ({direction} risk by {abs(row['shap_value']):.2f})")

# Waterfall plot for this prediction
shap.waterfall_plot(
    shap.Explanation(
        values=sample_shap,
        base_values=explainer.expected_value,
        data=X_val_scaled[high_risk_idx],
        feature_names=X_train.columns
    ),
    show=True
)

In [ ]:
# SHAP summary plot
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_val_scaled, feature_names=X_train.columns, show=False)
plt.title('SHAP Feature Importance')
plt.tight_layout()
plt.show()

In [ ]:
# Initialize SHAP explainer for linear models
explainer = shap.LinearExplainer(model_lr, X_train_scaled, feature_names=X_train.columns)

# Calculate SHAP values for validation set
print("Calculating SHAP values...")
shap_values = explainer.shap_values(X_val_scaled)

print(f"✓ SHAP values calculated")
print(f"  Shape: {shap_values.shape}")
print(f"  Base value: {explainer.expected_value:.2f}")

## 7. SHAP Explainability

In [ ]:
# Get feature coefficients (weights)
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'coefficient': model_lr.coef_,
    'abs_coefficient': np.abs(model_lr.coef_)
}).sort_values('abs_coefficient', ascending=False)

# Plot top 20 features
plt.figure(figsize=(12, 8))
top_features = feature_importance.head(20)
colors = ['red' if x < 0 else 'green' for x in top_features['coefficient']]
plt.barh(range(len(top_features)), top_features['coefficient'], color=colors)
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Coefficient (Impact on Risk Score)')
plt.title('Top 20 Most Important Features')
plt.axvline(0, color='black', linestyle='-', linewidth=0.8)
plt.tight_layout()
plt.show()

print("\nTop 10 risk-increasing features:")
print(feature_importance.head(10)[['feature', 'coefficient']].to_string(index=False))

print("\nTop 10 risk-decreasing features:")
print(feature_importance.tail(10)[['feature', 'coefficient']].to_string(index=False))

## 6. Feature Importance Analysis

In [ ]:
# Visualize predictions vs actuals
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].scatter(y_train_score, y_pred_train, alpha=0.5)
axes[0].plot([0, 100], [0, 100], 'r--', lw=2)
axes[0].set_xlabel('Actual Risk Score')
axes[0].set_ylabel('Predicted Risk Score')
axes[0].set_title(f'Training Set (R²={train_r2:.3f})')
axes[0].grid(True, alpha=0.3)

axes[1].scatter(y_val_score, y_pred_val, alpha=0.5, color='orange')
axes[1].plot([0, 100], [0, 100], 'r--', lw=2)
axes[1].set_xlabel('Actual Risk Score')
axes[1].set_ylabel('Predicted Risk Score')
axes[1].set_title(f'Validation Set (R²={val_r2:.3f})')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Feature scaling (important for logistic regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Train logistic regression for regression (predicting risk_score 0-100)
# We'll train a regressor and then map to risk levels
from sklearn.linear_model import Ridge

print("Training Logistic Regression model...")
model_lr = Ridge(alpha=1.0, random_state=RANDOM_STATE)
model_lr.fit(X_train_scaled, y_train_score)

# Make predictions
y_pred_train = model_lr.predict(X_train_scaled)
y_pred_val = model_lr.predict(X_val_scaled)

# Clip predictions to valid range [0, 100]
y_pred_train = np.clip(y_pred_train, 0, 100)
y_pred_val = np.clip(y_pred_val, 0, 100)

# Calculate metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

train_mse = mean_squared_error(y_train_score, y_pred_train)
train_mae = mean_absolute_error(y_train_score, y_pred_train)
train_r2 = r2_score(y_train_score, y_pred_train)

val_mse = mean_squared_error(y_val_score, y_pred_val)
val_mae = mean_absolute_error(y_val_score, y_pred_val)
val_r2 = r2_score(y_val_score, y_pred_val)

print(f"\n✓ Model trained successfully")
print(f"\n  Training Performance:")
print(f"    MSE: {train_mse:.2f}")
print(f"    MAE: {train_mae:.2f}")
print(f"    R²: {train_r2:.3f}")
print(f"\n  Validation Performance:")
print(f"    MSE: {val_mse:.2f}")
print(f"    MAE: {val_mae:.2f}")
print(f"    R²: {val_r2:.3f}")

## 5. Model Training - Logistic Regression (RECOMMENDED)

**Why Logistic Regression?**
- ✅ Highly interpretable (direct feature weights)
- ✅ Fast training and inference
- ✅ Easy to port to JavaScript for Forge Functions
- ✅ Works well with SHAP for explanations
- ✅ No external model files needed (just coefficients)

In [ ]:
# Visualize risk score distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(y_train_score, bins=30, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Risk Score')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Risk Score Distribution')
axes[0].axvline(30, color='green', linestyle='--', label='Low/Medium threshold')
axes[0].axvline(70, color='red', linestyle='--', label='Medium/High threshold')
axes[0].legend()

# Risk level counts
risk_counts = pd.Series(y_train_label).value_counts()
axes[1].bar(risk_counts.index, risk_counts.values, color=['green', 'orange', 'red'])
axes[1].set_xlabel('Risk Level')
axes[1].set_ylabel('Count')
axes[1].set_title('Risk Level Distribution')

plt.tight_layout()
plt.show()

# Feature correlation with risk score
correlations = X_train.corrwith(pd.Series(y_train_score, index=X_train.index)).sort_values(ascending=False)
print(f"\nTop 10 features correlated with risk score:")
print(correlations.head(10))

## 4. Exploratory Data Analysis

In [ ]:
# Initialize feature extractor
extractor = PRFeatureExtractor()

# Extract features for all samples
def extract_features_and_labels(raw_data):
    """Extract features and labels from raw PR data"""
    X_list = []
    y_scores = []
    y_labels = []
    
    for pr in raw_data:
        # Extract features
        features = extractor.extract_features(pr)
        X_list.append(features)
        
        # Extract labels
        y_scores.append(pr['labels']['risk_score'])
        y_labels.append(pr['labels']['risk_level'])
    
    # Convert to DataFrame
    X = pd.DataFrame(X_list)
    y_score = np.array(y_scores)
    y_label = np.array(y_labels)
    
    return X, y_score, y_label

# Extract training data
print("Extracting features from training data...")
X_train, y_train_score, y_train_label = extract_features_and_labels(train_raw)

# Extract validation data
print("Extracting features from validation data...")
X_val, y_val_score, y_val_label = extract_features_and_labels(val_raw)

print(f"\n✓ Feature extraction complete")
print(f"  - Feature matrix shape: {X_train.shape}")
print(f"  - Number of features: {X_train.shape[1]}")
print(f"  - Target range: {y_train_score.min():.1f} - {y_train_score.max():.1f}")

# Display sample features
print(f"\n  Sample features (first PR):")
for col in X_train.columns[:5]:
    print(f"    {col}: {X_train[col].iloc[0]:.2f}")

## 3. Feature Extraction

In [ ]:
# Load synthetic training data
with open(DATA_DIR / 'train_data.json', 'r') as f:
    train_raw = json.load(f)

with open(DATA_DIR / 'val_data.json', 'r') as f:
    val_raw = json.load(f)

print(f"✓ Data loaded")
print(f"  - Training samples: {len(train_raw)}")
print(f"  - Validation samples: {len(val_raw)}")

# Check distribution
train_labels = [d['labels']['risk_level'] for d in train_raw]
print(f"\n  Risk level distribution:")
print(f"    Low: {train_labels.count('low')} ({train_labels.count('low')/len(train_labels)*100:.1f}%)")
print(f"    Medium: {train_labels.count('medium')} ({train_labels.count('medium')/len(train_labels)*100:.1f}%)")
print(f"    High: {train_labels.count('high')} ({train_labels.count('high')/len(train_labels)*100:.1f}%)")

In [ ]:
# ⚠️ CONFIGURATION: Set your OpenAI API key if using embeddings
# NEVER commit this to version control! Use environment variables in production.
OPENAI_API_KEY = None  # Set to your key or use: os.getenv('OPENAI_API_KEY')

# Paths
DATA_DIR = Path('data')
MODEL_DIR = Path('models')
MODEL_DIR.mkdir(exist_ok=True)

# Model configuration
RANDOM_STATE = 42
TEST_SIZE = 0.2
USE_EMBEDDINGS = False  # Set to True if using historical similarity features

print(f"✓ Configuration loaded")
print(f"  - Random state: {RANDOM_STATE}")
print(f"  - Test size: {TEST_SIZE}")
print(f"  - Embeddings: {'Enabled' if USE_EMBEDDINGS else 'Disabled'}")

## 2. Configuration and Data Loading

In [ ]:
# Core libraries
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Machine learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, 
    f1_score, roc_auc_score, roc_curve, confusion_matrix
)
import joblib

# Explainability
import shap

# Optional: Advanced models
try:
    import lightgbm as lgb
    LIGHTGBM_AVAILABLE = True
except ImportError:
    LIGHTGBM_AVAILABLE = False
    print("LightGBM not installed. Only logistic regression will be available.")

# Optional: OpenAI for embeddings
try:
    from openai import OpenAI
    OPENAI_AVAILABLE = True
except ImportError:
    OPENAI_AVAILABLE = False
    print("OpenAI not installed. Historical similarity features will be disabled.")

# Feature extractor
import sys
sys.path.append('..')
from feature_extractor import PRFeatureExtractor

# Configuration
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries imported successfully")
print(f"  - LightGBM: {'Available' if LIGHTGBM_AVAILABLE else 'Not installed'}")
print(f"  - OpenAI: {'Available' if OPENAI_AVAILABLE else 'Not installed'}")

## 1. Import Required Libraries

# Code Risk Radar - Baseline Model Training

**Objective**: Train an explainable risk prediction model for Bitbucket pull requests.

**Approach**: 
- **Recommended**: Logistic Regression (highly interpretable, portable to JavaScript)
- **Alternative**: LightGBM (better accuracy, still explainable via SHAP)

**Key Features**:
- 6 risk axes: Complexity, Bug Probability, Security, Coupling, Volatility, Change Surface
- 39 engineered features total
- SHAP values for individual prediction explanations
- Deployable to Forge Functions or FastAPI microservice